Con la base de datos de toda la RMB voy a obtener dos nuevas: 
1. Sólo para el AMB. 
2. Sólo para el municipio de Barcelona.

In [1]:
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import pandas as pd

# Importo los archivos espaciales

In [2]:
#AMB
gdf_amb = gpd.read_file('G:/Mi unidad/Datos/españa/vectores/MUNICIPIOS-AMB-31N.gpkg')
gdf_amb = gdf_amb.to_crs(epsg=32631) #porque estaba mal seteado

In [3]:
#BCN
gdf_bcn = gpd.read_file('G:/Mi unidad/Datos/españa/vectores/MUNICIPOS-ESPAÑA-31N.gpkg')
gdf_bcn = gdf_bcn[gdf_bcn['NAMEUNIT'] == 'Barcelona']

gdf_bcn = gdf_bcn.to_crs(epsg=32631) #por si las dudas

# Importo el Excel y lo espacializo

In [4]:
df=pd.read_excel('data/primera_ronda/v-rmb-abril24.xlsx')
df=df.drop(columns=['ID'])

# Convertir el DataFrame a un GeoDataFrame
geometry = [Point(lon, lat) for lon, lat in zip(df['Lon_X'], df['Lat_Y'])]
crs = 'epsg:4326'  # Definir sistema de coordenadas WGS84
gdf_puntos = gpd.GeoDataFrame(df, crs=crs, geometry=geometry)

# Convertir el GeoDataFrame a EPSG:32631
gdf_puntos = gdf_puntos.to_crs(epsg=32631)

# Hago el spatial join

In [5]:
# Realizar una unión espacial entre los dos GeoDataFrames
bcn = gpd.sjoin(gdf_puntos, gdf_bcn[['geometry']], how='inner', predicate='intersects')

In [6]:
# Realizar una unión espacial entre los dos GeoDataFrames
amb = gpd.sjoin(gdf_puntos, gdf_amb[['geometry']], how='inner', predicate='intersects')

# Exporto

In [7]:
#Creo un ID
bcn.reset_index(drop=True, inplace=True)
bcn.index = bcn.index + 1
bcn.insert(0, 'ID', bcn.index)

#Exporto
bcn.drop(['geometry','index_right'],axis=1).to_excel('data/primera_ronda/v-mun_bcn-abril24.xlsx',index=False)

In [8]:
#Creo un ID
amb.reset_index(drop=True, inplace=True)
amb.index = amb.index + 1
amb.insert(0, 'ID', amb.index)

#Exporto
amb.drop(['geometry','index_right'],axis=1).to_excel('data/primera_ronda/v-amb-abril24.xlsx',index=False)

In [9]:
print("Longitud de datos para BCN:", len(bcn))
print("Longitud de datos para el AMB:", len(amb))
print("Longitud de datos para la RMB:", len(df))

Longitud de datos para BCN: 7177
Longitud de datos para el AMB: 14062
Longitud de datos para la RMB: 24700
